# MayAvi

In [ ]:
"""
To run the animation in a Jupyter notebook, you can use the following code:
> pip install mayavi
"""

from mayavi import mlab
from scipy.spatial import ConvexHull
import numpy as np
import math

def generate_points_data(num_points, scale):
    return np.random.rand(num_points, 3) * scale

class Faces():
    def __init__(self,tri, sig_dig=12, method="convexhull"):
        self.method=method
        self.tri = np.around(np.array(tri), sig_dig)
        self.grpinx = list(range(len(tri)))
        norms = np.around([self.norm(s) for s in self.tri], sig_dig)
        _, self.inv = np.unique(norms,return_inverse=True, axis=0)

    def norm(self,sq):
        cr = np.cross(sq[2]-sq[0],sq[1]-sq[0])
        return np.abs(cr/np.linalg.norm(cr))

    def isneighbor(self, tr1,tr2):
        a = np.concatenate((tr1,tr2), axis=0)
        return len(a) == len(np.unique(a, axis=0))+2

    def order(self, v):
        if len(v) <= 3:
            return v
        v = np.unique(v, axis=0)
        n = self.norm(v[:3])
        y = np.cross(n,v[1]-v[0])
        y = y/np.linalg.norm(y)
        c = np.dot(v, np.c_[v[1]-v[0],y])
        if self.method == "convexhull":
            h = ConvexHull(c)
            return v[h.vertices]
        else:
            mean = np.mean(c,axis=0)
            d = c-mean
            s = np.arctan2(d[:,0], d[:,1])
            return v[np.argsort(s)]

    def simplify(self):
        for i, tri1 in enumerate(self.tri):
            for j,tri2 in enumerate(self.tri):
                if j > i: 
                    if self.isneighbor(tri1,tri2) and \
                       self.inv[i]==self.inv[j]:
                        self.grpinx[j] = self.grpinx[i]
        groups = []
        for i in np.unique(self.grpinx):
            u = self.tri[self.grpinx == i]
            u = np.concatenate([d for d in u])
            u = self.order(u)
            groups.append(u)
        return groups

def calculate_camera_angles(num, rotate_axes, rotate_angle, camera_position):
    # Calculate base elev and azim from camera position
    x, y, z = camera_position
    r = math.sqrt(x**2 + y**2 + z**2)
    elev = math.degrees(math.asin(z / r))  # Base elevation angle
    azim = math.degrees(math.atan2(y, x))  # Base azimuthal angle

    # Apply rotation on top of base angles for each specified axis
    for axis in rotate_axes:
        if axis == 'x':
            elev += rotate_angle * num
        elif axis == 'y':
            azim += rotate_angle * num
        elif axis == 'z':
            elev += rotate_angle * num
            azim += rotate_angle * num

    return elev, azim

def plot_scatter(points, num, alpha):
    hull = ConvexHull(points)
    simplices = hull.simplices
    org_triangles = [points[s] for s in simplices]
    f = Faces(org_triangles)
    g = f.simplify()
    
    # Plot all points
    mlab.points3d(points[:, 0], points[:, 1], points[:, 2], color=(0, 0, 1), scale_factor=0.05)

    # Plot vertices of the convex hull
    mlab.points3d(hull.points[hull.vertices, 0], 
                  hull.points[hull.vertices, 1], 
                  hull.points[hull.vertices, 2], 
                  color=(1, 0, 0), 
                  scale_factor=0.1)

    # Plot the faces of the convex hull with adjusted opacity
    for face in g:
        mlab.triangular_mesh([v[0] for v in face], 
                             [v[1] for v in face], 
                             [v[2] for v in face], 
                             [[0, 1, 2]], 
                             color=(0, 1, 0), 
                             opacity=alpha)

    # Initialize text, positioned at the top with larger width
    #time_text = mlab.title('sss', size=32)
    #time_text.x_position = 0.5
    #time_text.y_position = 0.95
    # Update text
    
    time_text = mlab.text(0.01, 0.9, '', width=0.15)  # Increased width for larger text

    time_text.text = f'Time: {num}'


@mlab.animate(delay=100)
def anim(datasets, rotate_axes, rotate_angle, camera_position):
    f = mlab.gcf()
    text = mlab.text(0.01, 0.01, '', width=0.1)  # Initialize text, position it at bottom left

    while True:
        for num, points in enumerate(datasets):
            mlab.clf()
            plot_scatter(points, num, alpha=0.5)
            mlab.view(*calculate_camera_angles(num, rotate_axes, rotate_angle, camera_position))
            f.scene.render()
            yield

# Generate datasets
num_frames = 100
num_points = 100
scale = 2
datasets = [generate_points_data(num_points, scale) for _ in range(num_frames)]

# Run animation
anim(datasets, ['x', 'y'], 3, (1, 1, 1))

mlab.show()